In [11]:
import pandas as pd
import numpy as np
from deepchecks.tabular import Dataset
from deepchecks.tabular.suites import model_evaluation

In [21]:
!ls DAI_predict

default_model simple_model


In [22]:
!ls DAI_predict/simple_model

data---original-feature-importance.csv
h2oai_experiment_simple_model_custom_dataset_TitanicData2_dcTest_csv_predictions.csv
h2oai_experiment_simple_model_custom_dataset_TitanicData2_dcTrain_csv_predictions.csv


In [35]:
df_train = pd.read_csv('DAI_predict/simple_model/h2oai_experiment_simple_model_custom_dataset_TitanicData2_dcTrain_csv_predictions.csv')
df_test = pd.read_csv('DAI_predict/simple_model/h2oai_experiment_simple_model_custom_dataset_TitanicData2_dcTest_csv_predictions.csv')
df_train.shape, df_test.shape

((1047, 11), (262, 11))

In [36]:
df_train.head()

,Passenger_Id,pclass,sex,age,sibsp,parch,fare,survived,survived.0,survived.1,survived.predicted(th=0.40031)
0,648,3rd,female,22.0,0,0,7.7500,1,0.397761,0.602239,1
1,391,3rd,male,16.0,1,3,34.3750,0,0.874194,0.125806,0
2,351,1st,female,49.0,1,0,76.7292,1,0.140861,0.859139,1
3,1163,3rd,female,24.0,0,0,7.7500,0,0.397761,0.602239,1
4,105,3rd,female,18.0,0,0,9.8417,1,0.397761,0.602239,1


In [37]:
df_test.head()

,Passenger_Id,pclass,sex,age,sibsp,parch,fare,survived,survived.0,survived.1,survived.predicted(th=0.40031)
0,1298,1st,male,17.0,0,2,110.8833,1,0.528816,0.471184,1
1,575,1st,female,45.0,0,1,59.4000,1,0.056927,0.943073,1
2,328,3rd,male,20.0,0,0,7.8542,0,0.883736,0.116264,0
3,408,3rd,male,25.0,0,0,7.0500,0,0.883736,0.116264,0
4,844,1st,male,NaN,0,0,35.5000,1,0.636392,0.363608,0


In [38]:
df_fi = pd.read_csv('DAI_predict/simple_model/data---original-feature-importance.csv', skiprows=2)
df_fi

,label,value
0,sex,1.000
1,pclass,0.527
2,parch,0.000


In [54]:
def get_proba_array(df, proba_cols) -> np.array:
    ''' Return predict probability of classification problem as numpy array '''
    return df[proba_cols].to_numpy()

def get_feature_inportance_series(df, features) -> pd.Series:
    '''
    Get DAI original feature importance dataframe and return the importance values with feture name index.
    The order of feature (which is 'features') must be same as the order of training dataset features.
    ''' 
    return pd.merge(pd.DataFrame({'label':features}), df, on='label', how='left').fillna(0.).set_index('label')['value']

In [44]:
train_proba = get_proba_array(df_train, ['survived.0','survived.1'])
test_proba = get_proba_array(df_test, ['survived.0','survived.1'])
train_proba.shape, test_proba.shape

((1047, 2), (262, 2))

In [56]:
features = ['pclass','sex','age','sibsp','parch','fare']
fi = get_feature_inportance_series(df_fi, features)
fi

label
pclass    0.527
sex       1.000
age       0.000
sibsp     0.000
parch     0.000
fare      0.000
Name: value, dtype: float64

In [59]:
df_train

,Passenger_Id,pclass,sex,age,sibsp,parch,fare,survived,survived.0,survived.1,survived.predicted(th=0.40031)
0,648,3rd,female,22.0,0,0,7.7500,1,0.397761,0.602239,1
1,391,3rd,male,16.0,1,3,34.3750,0,0.874194,0.125806,0
2,351,1st,female,49.0,1,0,76.7292,1,0.140861,0.859139,1
3,1163,3rd,female,24.0,0,0,7.7500,0,0.397761,0.602239,1
4,105,3rd,female,18.0,0,0,9.8417,1,0.397761,0.602239,1
...,...,...,...,...,...,...,...,...,...,...,...
1042,1247,3rd,female,NaN,0,0,14.5000,0,0.397761,0.602239,1
1043,1114,3rd,male,20.0,0,0,4.0125,0,0.883736,0.116264,0
1044,862,1st,female,39.0,1,1,79.6500,1,0.061102,0.938898,1
1045,835,3rd,male,18.5,0,0,7.2292,0,0.883736,0.116264,0


In [61]:
train_features = ['Passenger_Id','pclass','sex','age','sibsp','parch','fare','survived']

ds_train = Dataset(df_train[train_features],
                   label='survived',
                   index_name='Passenger_Id',
                   cat_features=['pclass','sex'])
ds_test = Dataset(df_test[train_features],
                  label='survived',
                  index_name='Passenger_Id',
                  cat_features=['pclass','sex'])

In [64]:
type(ds_train)

deepchecks.tabular.dataset.Dataset

In [62]:
result = model_evaluation().run(train_dataset=ds_train,    # Train data (Dataset)
                                test_dataset=ds_test,      # Test data (Dataset)
                                feature_importance=fi,     # Feature importance (Series)
                                y_proba_train=train_proba, # Train prediction probability (array)
                                y_proba_test=test_proba)   # Test prediction probability (array)

/opt/anaconda3/envs/ML/lib/python3.10/site-packages/deepchecks/tabular/utils/feature_importance.py:454: UserWarning:

feature_importance does not sum to 1. Normalizing to 1.

deepchecks - WARNING - train and test datasets have common index - adding "train"/"test" prefixes. To avoid that provide datasets with no common indexes or pass the model object instead of the predictions.


deepchecks - WARNING - Could not find model's classes, using the observed classes. In order to make sure the classes used by the model are inferred correctly, please use the model_classes argument


In [63]:
result.show()

Accordion(children=(VBox(children=(HTML(value='\n<h1 id="summary_MT58CK1QZ9IH6EJUN0HCNI7OE">Model Evaluation S…